In [2]:
# Libraries
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
%matplotlib inline


In [3]:
from glob import glob
imagePatches = glob('../input/IDC_regular_ps50_idx5/**/*.png', recursive=True)

In [4]:
len(imagePatches)

277524

In [5]:
imagePatches[0]

'../input/IDC_regular_ps50_idx5/10295/0/10295_idx5_x1351_y1101_class0.png'

In [6]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [7]:
y = []
for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

In [8]:
images_df = pd.DataFrame()

In [9]:
images_df["images"] = imagePatches
images_df["labels"] = y

In [10]:
images_df.head()

,images,labels
0,../input/IDC_regular_ps50_idx5/10295/0/10295_i...,0
1,../input/IDC_regular_ps50_idx5/10295/0/10295_i...,0
2,../input/IDC_regular_ps50_idx5/10295/0/10295_i...,0
3,../input/IDC_regular_ps50_idx5/10295/0/10295_i...,0
4,../input/IDC_regular_ps50_idx5/10295/0/10295_i...,0


In [11]:
images_df.groupby("labels")["labels"].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [31]:
#Splitting data into train and val
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)
print("tranin dimension:",len(train))
print("validation dimension:",len(val))



tranin dimension: 222019
validation dimension: 55505


In [20]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (64,64))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [21]:
## Parameters for model

# Hyper parameters
num_epochs = 10
num_classes = 2
batch_size = 128
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [22]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [23]:
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [24]:
model = SimpleCNN().to(device)

In [25]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [26]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/1735], Loss: 0.3643
Epoch [1/10], Step [200/1735], Loss: 0.3546
Epoch [1/10], Step [300/1735], Loss: 0.3488
Epoch [1/10], Step [400/1735], Loss: 0.3788
Epoch [1/10], Step [500/1735], Loss: 0.3448
Epoch [1/10], Step [600/1735], Loss: 0.3726
Epoch [1/10], Step [700/1735], Loss: 0.3433
Epoch [1/10], Step [800/1735], Loss: 0.2979
Epoch [1/10], Step [900/1735], Loss: 0.2473
Epoch [1/10], Step [1000/1735], Loss: 0.3115
Epoch [1/10], Step [1100/1735], Loss: 0.3436
Epoch [1/10], Step [1200/1735], Loss: 0.2337
Epoch [1/10], Step [1300/1735], Loss: 0.2883
Epoch [1/10], Step [1400/1735], Loss: 0.2854
Epoch [1/10], Step [1500/1735], Loss: 0.3224
Epoch [1/10], Step [1600/1735], Loss: 0.3244
Epoch [1/10], Step [1700/1735], Loss: 0.2371
Epoch [2/10], Step [100/1735], Loss: 0.3272
Epoch [2/10], Step [200/1735], Loss: 0.2961
Epoch [2/10], Step [300/1735], Loss: 0.3538
Epoch [2/10], Step [400/1735], Loss: 0.3944
Epoch [2/10], Step [500/1735], Loss: 0.3034
Epoch [2/10], Step [600/

In [29]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy : {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy : 89.30907125484191 %
